In [1]:
import json
import numpy as np
import pandas as pd
from ppa.pars_cas import calc_stats, restructure_pars_cas
import os

In [2]:
locations = list(pd.read_csv('data/locations.csv').Location)

In [3]:
os.makedirs('docs/pars_wr', exist_ok=True)

In [11]:
def ext_targets(targets):
    pop = dict(targets[targets.Index == 'DrugTime'].set_index('Year').N)
    
    flatten_targets = dict()

    det = targets[targets.Index == "TxI"]
    det = {
        'Pub': dict(det[det.Tag == "Pub"].set_index('Year').N),
        'Eng': dict(det[det.Tag == "Pri"].set_index('Year').N),
    }

    for k, v in det['Pub'].items():
        flatten_targets[f'Det_Pub_{k}'] = {'m': v / pop[k], 'n': pop[k]}

    for k, v in det['Eng'].items():
        flatten_targets[f'Det_Eng_{k}'] = {'m': v / pop[k], 'n': pop[k]}

    prev = dict()

    for state in ['Asym', 'Sym', 'ExCS', 'Ut']:
        sel = targets[targets.Index == f"Prev{state}"]
        flatten_targets[f'Prev_{state}_2020'] = {'m': list(sel.M)[0], 'n': list(sel.N)[0]}

    return flatten_targets

In [19]:
for loc in locations:
    print(loc)
    os.makedirs(f'docs/pars_wr/{loc}', exist_ok=True)
    
    pars = json.load(open(f'data/{loc}/pars_cas.json', 'r'))
    inp = dict()
    inp['pars'] = ps = list()
    stats = list()
    
    for p in pars['pars']:
        stats.append(calc_stats(p))
        p = restructure_pars_cas(p)
        p = {k: v.tolist() if isinstance(v, np.ndarray) else v for k, v in p.items()}
        ps.append(p)

    stats = pd.DataFrame(stats)
    stats = stats.describe().T
    stats.to_csv(f'docs/pars_wr/{loc}/stats.csv')
    
    targets = pd.read_csv(f'data/{loc}/targets.csv')
    inp['targets'] = ext_targets(targets)
    
    with open(f'docs/pars_wr/{loc}/inputs.json', 'w') as f:
        json.dump(inp, f)
        
    

India
Andhra_Pradesh
Arunachal_Pradesh
Assam
Bihar
Chandigarh
Chhattisgarh
Delhi
Gujarat
Haryana
Himachal_Pradesh
Jammu_and_Kashmir
Jharkhand
Karnataka
Kerala
Madhya_Pradesh
Maharashtra
Manipur
Meghalaya
Nagaland
Odisha
Punjab
Rajasthan
Tamil_Nadu
Telangana
Uttar_Pradesh
Uttarakhand
West_Bengal
